In [ ]:
from robusttest.core.grid_time_series import GridTimeSeries as grid_ts
import warnings
import numpy as np
import pandas as pd

import plotly.graph_objects as go
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

In [ ]:
import pandapower as pp 
import simbench as sb

In [ ]:
single_zone_codes = sb.collect_all_simbench_codes(lv_level="", all_data=False)

In [ ]:
single_zone_codes

In [ ]:
grid = grid_ts('1-LV-rural1--0-sw', save_path='LV/1-LV-rural1--0-sw')

In [ ]:
pp.plotting.simple_plot(grid.net, respect_switches=True, bus_size=0.1)

In [ ]:
grid.plot()

In [ ]:
for sb_grid_code in single_zone_codes:
    grid = grid_ts(sb_grid_code=sb_grid_code)
    pp.plotting.simple_plot(grid.net, respect_switches=True, bus_size=0.1)

In [ ]:
variables = [("sgen", "p_mw"), ("load", "p_mw"), ("load", "q_mvar")]
grid.set_variables(variables=variables)

In [ ]:
grid.run_timeseries(variables)

In [ ]:
grid.create_ts()

In [ ]:
grid.values_line_ts_df.head()

In [ ]:
grid.create_measurements_ts()
grid.measurements_bus_ts_df.head()

In [ ]:
grid.measurements_line_ts_df.head()

In [ ]:
data_list, x_set_mean, x_set_std, edge_attr_set_mean, edge_attr_set_std = grid.create_pyg_data()

In [ ]:
import robusttest.interface.generate_LV_TS as glv
glv.generate_LV_TS()

In [ ]:
split_coef = 0.9
train_dataset = data_list[0:int(split_coef*len(data_list))]
test_dataset = data_list[int(split_coef*len(data_list)):]

In [ ]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
from robusttest.core.SE.gat_dsse import GAT_DSSE_Lightning
mu_v = 1e-1
reg_coefs = {
'mu_v': mu_v,
'mu_theta': mu_v,
'lam_v': 1e-4,
'lam_p': 1e-8,
'lam_pf': 1e-6,
'lam_reg': 1e2
}

hyperparameters = {
    'dim_nodes': 8,    # Example value
    'dim_lines': 6,
    'dim_out': 2,
    'dim_hid': 32,
    'gnn_layers': 8,
    'heads': 1,
    'K': 2,
    'dropout_rate': 0.3,
    'L': 5
}

model = GAT_DSSE_Lightning(hyperparameters, x_set_mean, x_set_std, edge_attr_set_mean, edge_attr_set_std, reg_coefs)

In [ ]:
from pytorch_lightning import Trainer
import torch


# Training with the Trainer
trainer = Trainer(max_epochs=50)
trainer.fit(model, train_loader)

In [ ]:
from robusttest.core.SE.gnn import GNN_SE

In [ ]:
gnn_se = GNN_SE(method='GNN', grid= grid)

In [ ]:
gnn_se.prepare_measurments()

In [ ]:
gnn_se.train()

In [ ]:
preds = gnn_se.predict(test= 15)

In [ ]:
len(preds[0])

In [ ]:
np.array(preds[0]).reshape(1,-1)[0]

In [ ]:
i = 0 
for idx, row in grid.results_ts_df.tail(50).iterrows():
    fig = go.Figure()
    fig.add_trace(go.Scatter(x= np.arange(59), y= row.values[:59], name= 'Truth'))
    fig.add_trace(go.Scatter(x= np.arange(59), y= np.array(preds[i]).reshape(1,-1)[0], name= 'est'))
    # fig.add_trace(go.Scatter(x= np.arange(59), y= pp_wls_se.results_df.values[idx], name= 'est'))
    fig.show()
    i += 1